<a href="https://colab.research.google.com/github/Pruthvik-Reddy/Warehouse_optimal_path/blob/main/Warehouse_path_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class Graph(object):

    def __init__(self, is_directed):
        self.all_edges = []
        self.edges=[]
        self.all_vertex = {}
        self.is_directed = is_directed

    def add_edge(self, id1,name1, id2,name2, weight=0):
        if id1 in self.all_vertex:
            vertex1 = self.all_vertex[id1]
        else:
            vertex1 = Vertex(id1,name1)
            self.all_vertex[id1] = vertex1


        if id2 in self.all_vertex:
            vertex2 = self.all_vertex[id2]
        else:
            vertex2 = Vertex(id2,name2)
            self.all_vertex[id2] = vertex2

        edge = Edge(vertex1, vertex2, self.is_directed, weight)
        self.all_edges.append(edge)
        self.edges.append(edge.values())
        vertex1.add_adjacent_vertex(edge, vertex2)
        if self.is_directed is not True:
            vertex2.add_adjacent_vertex(edge,vertex1)

 
class Edge(object):
    
    def __init__(self, vertex1, vertex2, is_directed, weight):
        self.vertex1 = vertex1
        self.vertex2 = vertex2
        self.is_directed = is_directed
        self.weight = weight

    def __eq__(self, other):
        return self.vertex1.id == other.vertex1.id and self.vertex2.id == other.vertex2.id

    def __hash(self):
        return hash(vertex1) + hash(vertex2)
    
    def __str__(self):
        return "Edge " + str(self.vertex1) + " " + str(self.vertex2) + " Weight-" + str(self.weight)
    def values(self):
      return (self.vertex1,self.vertex2,self.weight)

    def __repr__(self):
        return self.__str__()
    
class Vertex(object):

    def __init__(self, id,name):
        self.id = id
        self.name=name
        self.edges = []
        self.adjacent_vertices = []

    def add_adjacent_vertex(self, edge, vertex):
        self.edges.append(edge)
        self.adjacent_vertices.append(vertex)

    def get_degree(self):
        return len(self.edges)
    
    def __eq__(self, other):
        return self.id == other.id

    def __hash__(self):
        return hash(self.id)
    
    def __str__(self):
        return str("Vertex-" + str(self.name))
    
    def __repr__(self):
        return self.__str__();


    def __lt__(self, other):
        return self.id < other.id
                    
    def __gt__(self, other):
        return self.id > other.id
    


“Pick path” refers to the route a picker takes through the picking area to complete his or her picking tasks. Since this activity may account for 50% or more of warehouse/store operations, optimizing the pick path can provide quantifiable savings in operating costs.

In this algorithm, we use dijkstra algorithm to compute shortest paths.

A warehouse network can be interpreted as a giant graph in which every intersection is a node and every road is an edge.

Intersections->nodes
paths->edges

some edges are free-edges which mean that the width is good enough to accomodate multiple forklifts.
Remaining are non-free edges in which only one forklift is possible. So if one is going in one direction, there can be other forklift which follows it but there cannot be another froklift in the opposite direction.

Also, we have to take into consideration whether a particular edge has any forklifts in it picking items. In that case, we cannot give that edge for other forklifts for traversal.

Also, in case of drooping/picking items at a particular aisle, if there are forklifts at A30,A60 then A45 is not possible but A70 is possible.

Algorithm:(Not sure if correct,check the code too):
1. We maintain a data strcuture(called stack in the code) which stores the numbers (e.g 30,60) which checks if the destination is actually possible or not.(like if 30 and 60 in stack, 45 is not possible but 70 is possible). So if a path exists then it is important that we add that number to stack.

2. we shouldn't include any edges which have some forklifts picking/unloading in them. So we make weights of all such edges as infinity so that they won't be included in our graph.(we just clone the graph so that the original graph isn't affected)

3. Also after you get the path, we check for conflict in edges. We assume a speed and calculate the estimated time interval for a forklift in an aisle or edge.

4. If its a non-free edge and there is an overlap in time with other forklifts then we make changes by manipulating graphs i.e making the edge weights as infinity. So all the conflicting edges are made infinity and we calculate the shortest graph again.





In [ ]:
from heapq import *

class PriorityQueue(object):
    
    def __init__(self, is_min_heap):
        self.pq = []
        self.entry_finder = {}
        if(is_min_heap is True):
            self.mul = 1
        else :
            self.mul = -1
         
    def contains_task(self, task):
        if task in self.entry_finder:
            return True
        else:
            return False

    def get_task_priority(self, task):
        if task in self.entry_finder:
            return (self.entry_finder[task])[0]
        raise ValueError("task does not exist")
        
    def add_task(self, priority, task):
        if task in self.entry_finder:
            raise KeyError("Key already exists")
        entry = [self.mul*priority, False, task]
        self.entry_finder[task] = entry
        heappush(self.pq, entry)

    def change_task_priority(self, priority, task):
        if task not in self.entry_finder:
            raise KeyError("Task not found")
        self.remove_task(task)
        entry = [self.mul*priority, False, task]
        self.entry_finder[task] = entry
        heappush(self.pq, entry)
 
    def remove_task(self, task):
        entry = self.entry_finder.pop(task)
        entry[1] = True

    def pop_task(self):
        while self.pq:
            priority, removed, task = heappop(self.pq)
            if removed is False:
                del self.entry_finder[task]
                return task
        raise KeyError("pop from an empty priority queue")

    def peek_task(self):
        while self.pq:
            priority, removed, task = tuple(heappop(self.pq))
            if removed is False:
                 heappush(self.pq, [priority, False, task])
                 return task
        raise KeyError("pop from an empty priority queue")

    def is_empty(self):
        try:
            self.peek_task()
            return False
        except KeyError:
            return True

    def __str__(self):
        return str(self.entry_finder) + " " + str(self.pq)
        


In [ ]:

vertex_ids={
    "S_1":1,
    "S_2":2,
    "A_f":3,
    "A_b":4,
    "B_f":5,
     "B_b":6,
    "C_f":7,
    "C_b":8,
    "D_f":9,
    "D_b":10,
    "E_f":11,
    "E_b":12,
    "F_f":13,
    "F_b":14,
    "G_f":15,
    "G_b":16,
    "H_f":17,
    "H_b":18,
    "I_f":19,
    "I_b":20,
    "J_f":21,
    "J_b":22
}

In [ ]:

free_edges=set()
free_edges.add(("Vertex-S_1","Vertex-A_f"))
free_edges.add(("Vertex-A_f","Vertex-B_f"))
free_edges.add(("Vertex-B_f","Vertex-C_f"))
free_edges.add(("Vertex-C_f","Vertex-D_f"))
free_edges.add(("Vertex-D_f","Vertex-E_f"))
free_edges.add(("Vertex-E_f","Vertex-S_2"))

free_edges.add(("Vertex-S_2","Vertex-E_f"))
free_edges.add(("Vertex-A_f","Vertex-S_1"))
free_edges.add(("Vertex-B_f","Vertex-A_f"))
free_edges.add(("Vertex-C_f","Vertex-B_f"))
free_edges.add(("Vertex-D_f","Vertex-C_f"))
free_edges.add(("Vertex-E_f","Vertex-D_f"))

free_edges.add(("Vertex-A_f","Vertex-A_b"))
free_edges.add(("Vertex-E_f","Vertex-E_b"))

free_edges.add(("Vertex-A_b","Vertex-A_f"))
free_edges.add(("Vertex-E_b","Vertex-E_f"))



free_edges.add(("Vertex-A_b","Vertex-B_b"))
free_edges.add(("Vertex-B_b","Vertex-C_b"))
free_edges.add(("Vertex-C_b","Vertex-D_b"))
free_edges.add(("Vertex-D_b","Vertex-E_b"))

free_edges.add(("Vertex-B_b","Vertex-A_b"))
free_edges.add(("Vertex-C_b","Vertex-B_b"))
free_edges.add(("Vertex-D_b","Vertex-C_b"))
free_edges.add(("Vertex-E_b","Vertex-D_b"))


free_edges.add(("Vertex-A_b","Vertex-F_f"))
free_edges.add(("Vertex-E_b","Vertex-J_f"))

free_edges.add(("Vertex-F_f","Vertex-A_b"))
free_edges.add(("Vertex-J_f","Vertex-E_b"))



free_edges.add(("Vertex-F_f","Vertex-G_f"))
free_edges.add(("Vertex-G_f","Vertex-H_f"))
free_edges.add(("Vertex-H_f","Vertex-I_f"))
free_edges.add(("Vertex-I_f","Vertex-J_f"))

free_edges.add(("Vertex-G_f","Vertex-F_f"))
free_edges.add(("Vertex-H_f","Vertex-G_f"))
free_edges.add(("Vertex-I_f","Vertex-H_f"))
free_edges.add(("Vertex-J_f","Vertex-I_f"))


free_edges.add(("Vertex-F_f","Vertex-F_b"))
free_edges.add(("Vertex-J_f","Vertex-J_b"))

free_edges.add(("Vertex-F_b","Vertex-F_f"))
free_edges.add(("Vertex-J_b","Vertex-J_f"))

free_edges.add(("Vertex-B_b","Vertex-G_f"))
free_edges.add(("Vertex-C_b","Vertex-H_f"))
free_edges.add(("Vertex-D_b","Vertex-I_f"))


free_edges.add(("Vertex-G_f","Vertex-B_b"))
free_edges.add(("Vertex-H_f","Vertex-C_b"))
free_edges.add(("Vertex-I_f","Vertex-D_b"))



free_edges.add(("Vertex-F_b","Vertex-G_b"))
free_edges.add(("Vertex-G_b","Vertex-H_b"))
free_edges.add(("Vertex-H_b","Vertex-I_b"))
free_edges.add(("Vertex-I_b","Vertex-J_b"))

free_edges.add(("Vertex-G_b","Vertex-F_b"))
free_edges.add(("Vertex-H_b","Vertex-G_b"))
free_edges.add(("Vertex-I_b","Vertex-H_b"))
free_edges.add(("Vertex-J_b","Vertex-I_b"))


In [ ]:

non_free_edges=set()
non_free_edges.add(("Vertex-B_f","Vertex-B_b"))
non_free_edges.add(("Vertex-C_f","Vertex-C_b"))
non_free_edges.add(("Vertex-D_f","Vertex-D_b"))
non_free_edges.add(("Vertex-G_f","Vertex-G_b"))
non_free_edges.add(("Vertex-H_f","Vertex-H_b"))
non_free_edges.add(("Vertex-I_f","Vertex-I_b"))

non_free_edges.add(("Vertex-B_b","Vertex-B_f"))
non_free_edges.add(("Vertex-C_b","Vertex-C_f"))
non_free_edges.add(("Vertex-D_b","Vertex-D_f"))
non_free_edges.add(("Vertex-G_b","Vertex-G_f"))
non_free_edges.add(("Vertex-H_b","Vertex-H_f"))
non_free_edges.add(("Vertex-I_b","Vertex-I_f"))


In [ ]:


print(len(free_edges))
print(len(non_free_edges))

54
12


In [ ]:
from collections import defaultdict
stack=defaultdict(list)
print(stack["A"])

overlapping_dict=defaultdict(list)


[]


In [ ]:


import sys


def shortest_path(graph, sourceVertex):

    min_heap = PriorityQueue(True)
    distance = {}
    parent = {}

    for vertex in graph.all_vertex.values():
        min_heap.add_task(float('inf'), vertex)

    min_heap.change_task_priority(0, sourceVertex)
    distance[sourceVertex] = 0
    parent[sourceVertex] = None

    while min_heap.is_empty() is False:
        task = min_heap.peek_task()
        weight = min_heap.get_task_priority(task)               
        current =  min_heap.pop_task()
        distance[current] = weight

        for edge in current.edges:
            adjacent = get_other_vertex_for_edge(current, edge)
            if min_heap.contains_task(adjacent) is False:
                continue

            new_distance = distance[current] + edge.weight
            if min_heap.get_task_priority(adjacent) > new_distance:
                min_heap.change_task_priority(new_distance, adjacent)
                parent[adjacent] = current
                
                
    dist={}
    par={}
    for key,value in distance.items():
      a=key.__str__()
      dist[a]=value
    for key,value in parent.items():
      a=key.__str__()
      val=value.__str__()
      par[a]=val
    return dist,par


def get_other_vertex_for_edge(vertex, edge):
    if edge.vertex1.id == vertex.id:
        return edge.vertex2
    else:
        return edge.vertex1


graph = Graph(False)
graph.add_edge(1,"S_1",3,"A_f",4)
graph.add_edge(2,"S_2",11,"E_f",4)
graph.add_edge(3,"A_f",4,"A_b",100)
graph.add_edge(3,"A_f",5,"B_f",8)
graph.add_edge(5,"B_f",6,"B_b",100)
graph.add_edge(5,"B_f",7,"C_f",8)
graph.add_edge(7,"C_f",8,"C_b",100)
graph.add_edge(7,"C_f",9,"D_f",8)


graph.add_edge(9,"D_f",10,"D_b",100)
graph.add_edge(9,"D_f",11,"E_f",8)
graph.add_edge(11,"E_f",12,"E_b",100)


graph.add_edge(13,"F_f",14,"F_b",100)
graph.add_edge(13,"F_f",15,"G_f",8)
graph.add_edge(13,"F_f",4,"A_b",12)
graph.add_edge(15,"G_f",16,"G_b",100)

graph.add_edge(15,"G_f",17,"H_f",8)
graph.add_edge(17,"H_f",18,"H_b",100)
graph.add_edge(17,"H_f",19,"I_f",8)
graph.add_edge(19,"I_f",20,"I_b",100)
graph.add_edge(19,"I_f",21,"J_f",8)
graph.add_edge(6,"J_f",22,"J_b",100)

graph.add_edge(4,"A_b",6,"B_b",8)
graph.add_edge(6,"B_b",15,"G_f",12)
graph.add_edge(6,"B_b",8,"C_b",8)
graph.add_edge(8,"C_b",17,"H_f",12)
graph.add_edge(8,"C_b",10,"D_b",8)
graph.add_edge(10,"D_b",19,"I_f",12)
graph.add_edge(10,"D_b",12,"E_b",8)

graph.add_edge(12,"E_b",21,"J_f",12)
graph.add_edge(14,"F_b",16,"G_b",8)
graph.add_edge(16,"G_b",18,"H_b",8)
graph.add_edge(18,"H_b",20,"I_b",8)
graph.add_edge(20,"I_b",22,"J_b",8)






distance,parent= shortest_path(graph, graph.all_vertex[1])
print(parent["Vertex-D_b"])


Vertex-D_f


In [ ]:

def change_edges2(graph):
  g=Graph(False)
  for i in range(len(graph.edges)):
    start=graph.edges[i][0].__str__()[-3:]
    end=graph.edges[i][1].__str__()[-3:]
    if start[0]==end[0]:
      if stack[start[0]]!=[]:
        tup=graph.edges[i]
        id1=vertex_ids[start]
        id2=vertex_ids[end]

        g.add_edge(id1,start,id2,end,float('inf'))
      else:
        tup=graph.edges[i]
        id1=vertex_ids[start]
        id2=vertex_ids[end]
        g.add_edge(id1,start,id2,end,tup[2])


    else:
      tup=graph.edges[i]
      id1=vertex_ids[start]
      id2=vertex_ids[end]
      g.add_edge(id1,start,id2,end,tup[2])


        
  return g
  

In [ ]:
def change_overlapping_edges(graph,vertices):
  g=Graph(False)
  for i in range(len(graph.edges)):
    start=graph.edges[i][0].__str__()[-3:]
    end=graph.edges[i][1].__str__()[-3:]
    if start[0]==end[0]:
      if start[0] in vertices:
        id1=vertex_ids[start]
        id2=vertex_ids[end]
        g.add_edge(id1,start,id2,end,float('inf'))
      else:
        tup=graph.edges[i]
        id1=vertex_ids[start]
        id2=vertex_ids[end]
        g.add_edge(id1,start,id2,end,tup[2])

    else:
        tup=graph.edges[i]
        id1=vertex_ids[start]
        id2=vertex_ids[end]
        g.add_edge(id1,start,id2,end,tup[2])


    

        
  return g
  


In [ ]:
free_edges_and_weights=set()
non_free_edges_and_weights=set()

free_edges_and_weights_hash=dict()
non_free_edges_and_weights_hash=dict()


for i in graph.edges:
  a,b,w=i[0].__str__(),i[1].__str__(),i[2]
  if (a,b) in free_edges:
    free_edges_and_weights.add((a,b,w))
    free_edges_and_weights.add((b,a,w))
    free_edges_and_weights_hash[(a,b)]=(a,b,w)
    free_edges_and_weights_hash[(b,a)]=(b,a,w)
  elif (a,b) in non_free_edges:
    non_free_edges_and_weights.add((a,b,w))
    non_free_edges_and_weights.add((b,a,w))
    non_free_edges_and_weights_hash[(a,b)]=(a,b,w)
    non_free_edges_and_weights_hash[(b,a)]=(b,a,w)



In [ ]:

def calculate_time(weight):
  speed=4
  return (weight/speed) 

In [ ]:
def conflict_time_intervals(vertex,interval):
  for i in range(len(overlapping_dict[vertex])):
    interval_2=overlapping_dict[vertex][i]
    if interval[0]<=interval_2[1] and interval[1]>=interval_2[0]:
      return True
  return False

In [ ]:

def compute_time_intervals(dist,path):
  time_intervals=[]
  current_time=datetime.datetime.now()
  curr_weight=0
  for i in range(1,len(path)):
    edge_weight=distance[path[i]]
    time_taken=calculate_time(edge_weight-curr_weight)
    curr_weight=edge_weight
    time_intervals.append((current_time,current_time+timedelta(minutes=time_taken)))
    current_time=current_time+timedelta(seconds=time_taken)
  return time_intervals

In [ ]:
def update_overlapping_dict(time_intervals,path):
  for i in range(len(path)-1):
    tup=(path[i],path[i+1])
    if tup not in free_edges:
      #print(tup)
      vertex_1=path[i][-3:]
      vert_1=vertex_1[0]
      
      overlapping_dict[vert_1].append(time_intervals[i])
      


In [ ]:
def destination_overlapping_check(destination_tuple,vert_3,destination_interval):
  check=conflict_time_intervals(vert_3,destination_interval)
  if(check):
    print('Another forklift has to pass throught the Aisle.')
    wait_time=0
    for i in overlapping_dict:
      if vert_3==i:
        wait_time=overlapping_dict[i][-1][1]

    print("Get to",destination_tuple[0],"and wait till",wait_time)
    return True
  else:
    return False


In [ ]:

def path_cost2(new_location,start_pos,graph):
  temp=new_location[2]
  flag=1
  if (ord(temp)<=57) and (ord(temp)>=48):
    flag=0
  num=0
  if flag==0:
    num=int(new_location[1:3])
  else:
    num=int(new_location[1:2])

  id_1=vertex_ids[start_pos[-3:]]
  #start_pos="S_1"
  graph=change_edges2(graph)
  
  distance,parent= shortest_path(graph, graph.all_vertex[id_1])
  aisle=new_location[0]
  stack[aisle].remove(num)
  #print(distance)
  #print(parent)
  #print("stack aisle lis",stack[aisle])


  if stack[aisle]==[]:
    dist1=distance["Vertex-"+aisle+"_f"]
    dist2=distance["Vertex-"+aisle+"_b"]
    dist=min(dist1+num,dist2+100-num)
    last_vertex=""
    if dist1+num==dist:
      last_vertex="Vertex-"+aisle+"_f"
    else:
      last_vertex="Vertex-"+aisle+"_b"

    lis=[num]
    stack[aisle]=lis
    #print(last_vertex)
    print("The Distance to be travelled is {a}".format(a=dist))
    path=[]
    path.insert(0,last_vertex)
    while(True):
      if parent[last_vertex]=="None":
        break
      path.insert(0,parent[last_vertex])
      last_vertex=parent[last_vertex]
    path.append(path[-1]+"--->"+new_location)
    print("Path is ",path)

    return dist,path
  lis=stack[aisle]
  
  if num>=lis[0] and num<=lis[-1]:
    return float('inf'),"Not Possible at the moment"

  elif num<lis[0]:
    lis=stack[aisle]
    lis.append(num)
    lis.sort()
    stack[aisle]=lis
  
    dist1=distance["Vertex-"+aisle+"_f"]
    dist=dist1+num
    last_vertex=""
    last_vertex="Vertex-"+aisle+"_f"
    print("The Distance to be travelled is {a}".format(a=dist))
    path=[]
    path.insert(0,last_vertex)
    while(True):
      if parent[last_vertex]=="None":
        break
      path.insert(0,parent[last_vertex])
      last_vertex=parent[last_vertex]
    path.append(path[-1]+"--->"+new_location)
    print("Path is ",path)
    
    return dist,path

  elif num>lis[-1]:
    lis=stack[aisle]
    lis.append(num)
    lis.sort()
    stack[aisle]=lis
  
    dist2=distance["Vertex-"+aisle+"_b"]
    dist=dist2+100-num
    last_vertex=""
    last_vertex="Vertex-"+aisle+"_b"


    print("The Distance to be travelled is {a}".format(a=dist))
    path=[]
    path.insert(0,last_vertex)
    while(True):
      if parent[last_vertex]=="None":
        break
      path.insert(0,parent[last_vertex])
      last_vertex=parent[last_vertex]
    path.append(path[-1]+"--->"+new_location)
    print("Path is ",path)

    return dist,path
  
    

In [ ]:
import datetime
from datetime import timedelta
def check_overlapping(new_location,start_pos,graph,dist,path):
  #print("In check overlapping")
  #print(dist)
  #print(path)
  time_intervals=[]
  current_time=datetime.datetime.now()
  curr_weight=0
  for i in range(1,len(path)):
    edge_weight=distance[path[i]]
    time_taken=calculate_time(edge_weight-curr_weight)
    curr_weight=edge_weight

    time_intervals.append((current_time,current_time+timedelta(minutes=time_taken)))
    current_time=current_time+timedelta(seconds=time_taken)
  #print(time_intervals)
  


  vert_3=path[-1][-3:][0]
  if path[-1][-3:][-1]=="f":
    destination_tuple=(path[-1],"Vertex-"+vert_3+"_"+"b")
  else:
    destination_tuple=(path[-1],"Vertex-"+vert_3+"_"+"f")
  for i in graph.edges:
    
    if (i[0].__str__(),i[1].__str__())==destination_tuple or (i[1].__str__(),i[0].__str__())==destination_tuple:
      destination_weight=i[2]
  dest_time=calculate_time(destination_weight)
  if dest_time!=float('inf'):
    destination_interval=(time_intervals[-1][1],time_intervals[-1][1]+timedelta(minutes=dest_time))

    
  conflicting_aisles=set()
  for i in range(len(path)-1):
    tup=(path[i],path[i+1])
    if tup not in free_edges:
      vertex_1=path[i][-3:]
      vert_1=vertex_1[0]
      if (overlapping_dict[vert_1]==[]):
        overlapping_dict[vert_1].append(time_intervals[i])
        if dest_time!=float('inf'):
          dest_check=destination_overlapping_check(destination_tuple,vert_3,destination_interval)
        return dist,path,False
      elif not (conflict_time_intervals(vert_1,time_intervals[i])):
        overloverlapping_dict[vert_1].append(time_intervals[i])
        if dest_time!=float('inf'):
          dest_check=destination_overlapping_check(destination_tuple,vert_3,destination_interval)
        
        return dist,path,False
      elif (conflict_time_intervals(vert_1,time_intervals[i])):
        conflicting_aisles.add(vert_1)
  
  #print(conflicting_aisles)    
  if (len(conflicting_aisles)>0):    
    g=change_overlapping_edges(graph,conflicting_aisles)
    #print(g.edges)
    dist,path=path_cost2(new_location,start_pos,g)
    n_4=len(path)-1
    time_intervals=compute_time_intervals(dist,path[:n_4])
    #print(overlapping_dict)
    update_overlapping_dict(time_intervals,path[:n_4])
    #print(overlapping_dict)
    if dest_time!=float('inf'):
      dest_check=destination_overlapping_check(destination_tuple,vert_3,destination_interval)
        
    return dist,path,True

  if dest_time!=float('inf'):
      dest_check=destination_overlapping_check(destination_tuple,vert_3,destination_interval)
          
  

  return dist,path,False

In [ ]:


def path_cost(new_location,start_pos,graph):
  temp=new_location[2]
  flag=1
  if (ord(temp)<=57) and (ord(temp)>=48):
    flag=0
  num=0
  if flag==0:
    num=int(new_location[1:3])
  else:
    num=int(new_location[1:2])

  id_1=vertex_ids[start_pos[-3:]]
  #start_pos="S_1"
  graph=change_edges2(graph)
  
  distance,parent= shortest_path(graph, graph.all_vertex[id_1])
  aisle=new_location[0]


  

  if stack[aisle]==[]:
    dist1=distance["Vertex-"+aisle+"_f"]
    dist2=distance["Vertex-"+aisle+"_b"]
    dist=min(dist1+num,dist2+100-num)
    last_vertex=""
    if dist1+num==dist:
      last_vertex="Vertex-"+aisle+"_f"
    else:
      last_vertex="Vertex-"+aisle+"_b"

    lis=[num]
    stack[aisle]=lis
    #print(last_vertex)
    print("The Distance to be travelled is {a}".format(a=dist))
    path=[]
    path.insert(0,last_vertex)
    while(True):
      if parent[last_vertex]=="None":
        break
      path.insert(0,parent[last_vertex])
      last_vertex=parent[last_vertex]
    path.append(path[-1]+"--->"+new_location)
    print("Path is ",path)

    #Checking overlaps
    #print("Overlapping ---")
    n_3=len(path)-1

    


    dist_1,path_1,flag=check_overlapping(new_location,start_pos,graph,distance,path[:n_3])
    #print(dist_1)
    #print(path_1)
    #print("End ----")
    if flag:
      return dist_1,path_1
    return dist,path
    #return dist,path
  lis=stack[aisle]
  
  if num>=lis[0] and num<=lis[-1]:
    return float('inf'),"Not Possible at the moment"

  elif num<lis[0]:
    lis=stack[aisle]
    lis.append(num)
    lis.sort()
    stack[aisle]=lis
  
    dist1=distance["Vertex-"+aisle+"_f"]
    dist=dist1+num
    last_vertex=""
    last_vertex="Vertex-"+aisle+"_f"
    print("The Distance to be travelled is {a}".format(a=dist))
    path=[]
    path.insert(0,last_vertex)
    while(True):
      if parent[last_vertex]=="None":
        break
      path.insert(0,parent[last_vertex])
      last_vertex=parent[last_vertex]
    path.append(path[-1]+"--->"+new_location)
    print("Path is ",path)
    
    #Checking overlaps
    #print("Overlapping ---")
    
    n_3=len(path)-1


    

    dist_1,path_1,flag=check_overlapping(new_location,start_pos,graph,distance,path[:n_3])
    #print(dist_1)
    #print(path_1)
    #print("End ----")
    if flag:
      return dist_1,path_1
    return dist,path
    #return dist,path

  elif num>lis[-1]:
    lis=stack[aisle]
    lis.append(num)
    lis.sort()
    stack[aisle]=lis
  
    dist2=distance["Vertex-"+aisle+"_b"]
    dist=dist2+100-num
    last_vertex=""
    last_vertex="Vertex-"+aisle+"_b"


    print("The Distance to be travelled is {a}".format(a=dist))
    path=[]
    path.insert(0,last_vertex)
    while(True):
      if parent[last_vertex]=="None":
        break
      path.insert(0,parent[last_vertex])
      last_vertex=parent[last_vertex]
    path.append(path[-1]+"--->"+new_location)
    print("Path is ",path)

    #Checking overlaps
    #print("Overlapping ---")
    
    n_3=len(path)-1


    


    dist_1,path_1,flag=check_overlapping(new_location,start_pos,graph,distance,path[:n_3])
    #print(dist_1)
    #print(path_1)
    #print("End ----")
    if flag:
      return dist_1,path_1
    return dist,path
    
    #return dist,path
  
    

    




In [ ]:
def get_path_from_present_location(present_location,forklift_name,graph,dest_location):
  #present_location["forklift_1"]="Vertex-S_1"
  
  if(len(present_location[forklift_name])<9):
    new_location=present_location[forklift_name]
    temp=present_location[forklift_name][2]
    flag=1
    if (ord(temp)<=57) and (ord(temp)>=48):
      flag=0
    num=0
    if flag==0:
      num=int(new_location[1:3])
    else:
      num=int(new_location[1:2])
    aisle=new_location[0]
    if stack[aisle]!=[]:
      lis=stack[aisle]
  
      if num>lis[0] and num<lis[-1]:
        return float('inf'),"Not Possible at the moment"
      elif num<=lis[0]:
        lis=stack[aisle]
        lis.remove(num)
        stack[aisle]=lis
      
        present_location[forklift_name]="Vertex-"+aisle+"_f"
        
      elif num>=lis[-1]:
        lis=stack[aisle]
        lis.remove(num)
        stack[aisle]=lis
      
        present_location[forklift_name]="Vertex-"+aisle+"_b"

  distance,parent= shortest_path(graph, graph.all_vertex[ids[forklift_name]])
  #dest_location="A60C"
  dist,paths=path_cost(dest_location,present_location[forklift_name],graph)
  edge=""
  for i in graph.edges:
    ind0=i[0].__str__()
    ind1=i[1].__str__()
    if(ind0==paths[0] and ind1==paths[1]) or (ind1==paths[0] and ind0==paths[1]):
      edge=(ind0,ind1,i[2])
  return dist,paths


In [ ]:

import datetime

def calculate_time_difference(time1,time2):
  fmt = '%Y-%m-%d %H:%M:%S'
  tstamp1 = datetime.datetime.strptime(time1, fmt)
  tstamp2 = datetime.datetime.strptime(time2, fmt)

  if tstamp1 > tstamp2:
      td = tstamp1 - tstamp2
  else:
      td = tstamp2 - tstamp1
  td_mins = int(round(td.total_seconds() / 60))

  return td_mins

In [ ]:
print(stack)

defaultdict(<class 'list'>, {'A': []})


In [ ]:
present_location={}
ids={}
dist={}
paths={}
forklift_name="forklift_1"

present_location["forklift_1"]="Vertex-S_1"
  
for key,value in graph.all_vertex.items():
  val=value.__str__()
  if val==present_location[forklift_name]:
    ids[forklift_name]=key

dist_,path_=get_path_from_present_location(present_location,forklift_name,graph,"D60C")
edges=[]
print(path_)

dist[forklift_name]=dist_
paths[forklift_name]=path_



The Distance to be travelled is 88
Path is  ['Vertex-S_1', 'Vertex-A_f', 'Vertex-B_f', 'Vertex-C_f', 'Vertex-D_f', 'Vertex-D_f--->D60C']
['Vertex-S_1', 'Vertex-A_f', 'Vertex-B_f', 'Vertex-C_f', 'Vertex-D_f', 'Vertex-D_f--->D60C']


In [ ]:
forklift_name="forklift_2"

present_location["forklift_2"]="Vertex-S_1"
  
for key,value in graph.all_vertex.items():
  val=value.__str__()
  if val==present_location[forklift_name]:
    ids[forklift_name]=key

dist_,path_=get_path_from_present_location(present_location,forklift_name,graph,"D20C")
dist[forklift_name]=dist_
print(path_)
paths[forklift_name]=path_

edges=[]
print(path_)
dist[forklift_name]=dist_
paths[forklift_name]=path_



The Distance to be travelled is 48
Path is  ['Vertex-S_1', 'Vertex-A_f', 'Vertex-B_f', 'Vertex-C_f', 'Vertex-D_f', 'Vertex-D_f--->D20C']
['Vertex-S_1', 'Vertex-A_f', 'Vertex-B_f', 'Vertex-C_f', 'Vertex-D_f', 'Vertex-D_f--->D20C']
['Vertex-S_1', 'Vertex-A_f', 'Vertex-B_f', 'Vertex-C_f', 'Vertex-D_f', 'Vertex-D_f--->D20C']


In [ ]:
print(overlapping_dict)

defaultdict(<class 'list'>, {'D': []})


In [ ]:
forklift_name="forklift_3"

present_location["forklift_3"]="Vertex-S_1"
  
for key,value in graph.all_vertex.items():
  val=value.__str__()
  if val==present_location[forklift_name]:
    ids[forklift_name]=key

dist_,path_=get_path_from_present_location(present_location,forklift_name,graph,"D70C")
dist[forklift_name]=dist_
print(path_)
paths[forklift_name]=path_

edges=[]
print(path_)
dist[forklift_name]=dist_
paths[forklift_name]=path_


The Distance to be travelled is 158
Path is  ['Vertex-S_1', 'Vertex-A_f', 'Vertex-B_f', 'Vertex-C_f', 'Vertex-C_b', 'Vertex-D_b', 'Vertex-D_b--->D70C']
['Vertex-S_1', 'Vertex-A_f', 'Vertex-B_f', 'Vertex-C_f', 'Vertex-C_b', 'Vertex-D_b', 'Vertex-D_b--->D70C']
['Vertex-S_1', 'Vertex-A_f', 'Vertex-B_f', 'Vertex-C_f', 'Vertex-C_b', 'Vertex-D_b', 'Vertex-D_b--->D70C']


In [ ]:
print(overlapping_dict)

defaultdict(<class 'list'>, {'D': [], 'C': [(datetime.datetime(2021, 6, 2, 10, 9, 0, 360767), datetime.datetime(2021, 6, 2, 10, 34, 0, 360767))]})


In [ ]:
forklift_name="forklift_4"

present_location["forklift_4"]="Vertex-S_1"
  
for key,value in graph.all_vertex.items():
  val=value.__str__()
  if val==present_location[forklift_name]:
    ids[forklift_name]=key

dist_,path_=get_path_from_present_location(present_location,forklift_name,graph,"D65C")
dist[forklift_name]=dist_
print(path_)
paths[forklift_name]=path_

edges=[]
print(path_)
dist[forklift_name]=dist_
paths[forklift_name]=path_


Not Possible at the moment
Not Possible at the moment


In [ ]:
forklift_name="forklift_5"

present_location["forklift_5"]="Vertex-S_1"
  
for key,value in graph.all_vertex.items():
  val=value.__str__()
  if val==present_location[forklift_name]:
    ids[forklift_name]=key

dist_,path_=get_path_from_present_location(present_location,forklift_name,graph,"H20C")
dist[forklift_name]=dist_
print(path_)
paths[forklift_name]=path_

edges=[]
print(path_)
dist[forklift_name]=dist_
paths[forklift_name]=path_


The Distance to be travelled is 152
Path is  ['Vertex-S_1', 'Vertex-A_f', 'Vertex-B_f', 'Vertex-C_f', 'Vertex-C_b', 'Vertex-H_f', 'Vertex-H_f--->H20C']
The Distance to be travelled is 152
Path is  ['Vertex-S_1', 'Vertex-A_f', 'Vertex-B_f', 'Vertex-B_b', 'Vertex-C_b', 'Vertex-H_f', 'Vertex-H_f--->H20C']
['Vertex-S_1', 'Vertex-A_f', 'Vertex-B_f', 'Vertex-B_b', 'Vertex-C_b', 'Vertex-H_f', 'Vertex-H_f--->H20C']
['Vertex-S_1', 'Vertex-A_f', 'Vertex-B_f', 'Vertex-B_b', 'Vertex-C_b', 'Vertex-H_f', 'Vertex-H_f--->H20C']


In [ ]:
print(overlapping_dict)

defaultdict(<class 'list'>, {'D': [], 'C': [(datetime.datetime(2021, 6, 4, 8, 9, 27, 645953), datetime.datetime(2021, 6, 4, 8, 34, 27, 645953))], 'B': [(datetime.datetime(2021, 6, 4, 8, 9, 40, 126180), datetime.datetime(2021, 6, 4, 8, 34, 40, 126180))], 'H': []})


In [ ]:
forklift_name="forklift_5"

present_location["forklift_5"]="Vertex-S_1"
  
for key,value in graph.all_vertex.items():
  val=value.__str__()
  if val==present_location[forklift_name]:
    ids[forklift_name]=key

dist_,path_=get_path_from_present_location(present_location,forklift_name,graph,"C20C")
dist[forklift_name]=dist_
print(path_)
paths[forklift_name]=path_

edges=[]
print(path_)
dist[forklift_name]=dist_
paths[forklift_name]=path_


The Distance to be travelled is 40
Path is  ['Vertex-S_1', 'Vertex-A_f', 'Vertex-B_f', 'Vertex-C_f', 'Vertex-C_f--->C20C']
Another forklift has to pass throught the Aisle.
Get to Vertex-C_f and wait till 2021-06-07 05:14:16.818769
['Vertex-S_1', 'Vertex-A_f', 'Vertex-B_f', 'Vertex-C_f', 'Vertex-C_f--->C20C']
['Vertex-S_1', 'Vertex-A_f', 'Vertex-B_f', 'Vertex-C_f', 'Vertex-C_f--->C20C']


In [ ]:
forklift_name="forklift_7"

present_location["forklift_7"]="Vertex-S_1"
  
for key,value in graph.all_vertex.items():
  val=value.__str__()
  if val==present_location[forklift_name]:
    ids[forklift_name]=key

dist_,path_=get_path_from_present_location(present_location,forklift_name,graph,"C56C")
dist[forklift_name]=dist_
print(path_)
paths[forklift_name]=path_

edges=[]
print(path_)
dist[forklift_name]=dist_
paths[forklift_name]=path_


Not Possible at the moment
Not Possible at the moment


In [ ]:
forklift_name="forklift_3"

present_location["forklift_3"]="D70C"
  
for key,value in graph.all_vertex.items():
  val=value.__str__()
  if val==present_location[forklift_name]:
    ids[forklift_name]=key

dist_,path_=get_path_from_present_location(present_location,forklift_name,graph,"H30C")
dist[forklift_name]=dist_
print(path_)
paths[forklift_name]=path_

edges=[]
print(path_)
dist[forklift_name]=dist_
paths[forklift_name]=path_


The Distance to be travelled is 190
Path is  ['Vertex-D_b', 'Vertex-I_f', 'Vertex-I_b', 'Vertex-H_b', 'Vertex-H_b--->H30C']
['Vertex-D_b', 'Vertex-I_f', 'Vertex-I_b', 'Vertex-H_b', 'Vertex-H_b--->H30C']
['Vertex-D_b', 'Vertex-I_f', 'Vertex-I_b', 'Vertex-H_b', 'Vertex-H_b--->H30C']


In [ ]:
forklift_name="forklift_2"

present_location["forklift_2"]="Vertex-S_1"
  
for key,value in graph.all_vertex.items():
  val=value.__str__()
  if val==present_location[forklift_name]:
    ids[forklift_name]=key

dist_,path_=get_path_from_present_location(present_location,forklift_name,graph,"G20C")
dist[forklift_name]=dist_
print(path_)
paths[forklift_name]=path_

edges=[]
print(path_)
dist[forklift_name]=dist_
paths[forklift_name]=path_


The Distance to be travelled is 144
Path is  ['Vertex-S_1', 'Vertex-A_f', 'Vertex-B_f', 'Vertex-B_b', 'Vertex-G_f', 'Vertex-G_f--->G20C']
In check overlapping
{'Vertex-S_1': 0, 'Vertex-A_f': 4, 'Vertex-B_f': 12, 'Vertex-C_f': 20, 'Vertex-D_f': 28, 'Vertex-E_f': 36, 'Vertex-S_2': 40, 'Vertex-A_b': 104, 'Vertex-B_b': 112, 'Vertex-F_f': 116, 'Vertex-C_b': 120, 'Vertex-G_f': 124, 'Vertex-D_b': 128, 'Vertex-H_f': 132, 'Vertex-E_b': 136, 'Vertex-I_f': 140, 'Vertex-J_f': 148, 'Vertex-J_b': 212, 'Vertex-F_b': 216, 'Vertex-I_b': 220, 'Vertex-G_b': 224, 'Vertex-H_b': 228}
['Vertex-S_1', 'Vertex-A_f', 'Vertex-B_f', 'Vertex-B_b', 'Vertex-G_f']
[(datetime.datetime(2021, 5, 31, 20, 22, 35, 87103), datetime.datetime(2021, 5, 31, 20, 22, 36, 87103)), (datetime.datetime(2021, 5, 31, 20, 22, 36, 87103), datetime.datetime(2021, 5, 31, 20, 22, 38, 87103)), (datetime.datetime(2021, 5, 31, 20, 22, 38, 87103), datetime.datetime(2021, 5, 31, 20, 23, 3, 87103)), (datetime.datetime(2021, 5, 31, 20, 23, 3, 87103

In [ ]:
forklift_name="forklift_4"

present_location["forklift_4"]="Vertex-S_1"
  
for key,value in graph.all_vertex.items():
  val=value.__str__()
  if val==present_location[forklift_name]:
    ids[forklift_name]=key

dist_,path_=get_path_from_present_location(present_location,forklift_name,graph,"D65C")
dist[forklift_name]=dist_
print(path_)
paths[forklift_name]=path_

edges=[]
print(path_)
dist[forklift_name]=dist_
paths[forklift_name]=path_


The Distance to be travelled is 163
Path is  ['Vertex-S_1', 'Vertex-A_f', 'Vertex-B_f', 'Vertex-C_f', 'Vertex-C_b', 'Vertex-D_b', 'Vertex-D_b--->D65C']
['Vertex-S_1', 'Vertex-A_f', 'Vertex-B_f', 'Vertex-C_f', 'Vertex-C_b', 'Vertex-D_b', 'Vertex-D_b--->D65C']
['Vertex-S_1', 'Vertex-A_f', 'Vertex-B_f', 'Vertex-C_f', 'Vertex-C_b', 'Vertex-D_b', 'Vertex-D_b--->D65C']


In [ ]:
print(stack)

defaultdict(<class 'list'>, {'A': [], 'B': [], 'C': [], 'D': [20, 60, 70], 'E': [], 'F': [], 'G': [10, 20], 'H': [], 'I': [], 'J': [20]})


In [ ]:
print(overlapping_dict)

defaultdict(<class 'list'>, {'B': [(datetime.datetime(2021, 6, 1, 12, 28, 44, 384955), datetime.datetime(2021, 6, 1, 12, 53, 44, 384955))]})
